In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *

In [2]:
steps = 5000
n_paths = 100000

In [3]:
x0 = 1.0 # initial condition
sigma = 1 # volatility
r = 0.01 # risk free rate
batch_size = 100 # batch size
T = 1 # maturity
dt = T/steps # mesh size

In [4]:
def generate_t(T, batch_size = batch_size):
    '''
    generate time steps
    dim = batch_size * (steps+1). eg. 2000 * 101
    '''
    return np.tile(np.linspace(1e-8, T+1e-8, steps + 1, dtype = np.float32), (batch_size,1))

In [5]:
t = generate_t(T, batch_size)[0][0:]
len(t)

5001

In [6]:
# Create GBM Paths from Brownian Motion Paths.
def Create_Paths(seed = 666):
    '''
    input: seed
    output: 
            (1) paths: geometric BM paths.
                dim = (n_paths * (steps+1))
            (2) W: BM paths.
                dim = (n_paths * (steps+1))
    '''

    
    # Generate BMs.
    np.random.seed(seed)
    dW = np.sqrt(dt)*np.random.normal(size=(n_paths, steps))
    W = np.concatenate((np.tile(1e-6, (n_paths, 1)), 
                        np.cumsum(dW, axis = 1, dtype = np.float32)), axis = 1)
    
    # initialization
    x = np.tile(x0, (n_paths, 1))
    paths = np.tile(x0, (n_paths, 1))

    # Generate GBMs. Euler scheme.
    for k in tqdm(range(steps)):
        x += r * x * dt + sigma * x * (W[:,k+1:k+2] - W[:,k:k+1])
        paths = np.concatenate((paths, x), axis = 1)
    paths = np.array(paths, dtype=np.float32)    

    return {'paths': paths, 'W': W}

In [7]:
Paths_and_W = Create_Paths()
Paths, W = Paths_and_W['paths'], Paths_and_W['W']
np.save("Paths_{}.npy".format(steps), Paths)
np.save("W_{}.npy".format(steps), W)

  8%|██████▍                                                                        | 404/5000 [00:20<03:49, 20.01it/s]


KeyboardInterrupt: 

In [ ]:
# plot GBM
for i in range(100):
    plt.plot(t, Paths[i])

In [ ]:
# plot BM
for i in range(100):
    plt.plot(t, W[i])